# LangGraph 기본 문법

## TypedDict

1. `dict`와 `TypedDict`의 주요 차이점:

   a) 타입 검사:
      - `dict`: 런타임에 타입 검사를 하지 않음
      - `TypedDict`: 정적 타입 검사 제공. 즉, 코드 작성 시 IDE나 타입 체커가 오류를 미리 잡아낼 수 있음

   b) 키와 값의 타입:
      - `dict`: 키와 값의 타입을 일반적으로 지정합니다 (예: Dict[str, str])
      - `TypedDict`: 각 키에 대해 구체적인 타입을 지정할 수 있음

   c) 유연성:
      - `dict`: 런타임에 키를 추가하거나 제거할 수 있음
      - `TypedDict`: 정의된 구조를 따라야 함. 추가적인 키는 타입 오류 발생시킴

2. `TypedDict`가 `dict` 대신 사용되는 이유:

   a) 타입 안정성: 더 엄격한 타입 검사를 제공하여 잠재적인 버그를 미리 방지할 수 있음

   b) 코드 가독성: 딕셔너리의 구조를 명확하게 정의할 수 있어 코드의 가독성 향상

   c) IDE 지원: IDE에서 자동 완성 및 타입 힌트를 더 정확하게 제공받을 수 있음

   d) 문서화: 코드 자체가 문서의 역할을 하여 딕셔너리의 구조를 명확히 보여줌

In [1]:
from typing import Dict, TypedDict

In [ ]:
# Dict 기반 정의
sample_dict: Dict[str, str] = {
    "name": "김영희",
    "age": "25",
    "job": "ai engineer",
}


# TypedDict
class Person(TypedDict):
    name: str
    age: int
    job: str


typed_dict: Person = {
    "name": "김영희",
    "age": 25,
    "job": "ai engineer",
}

In [ ]:
# Dict
sample_dict["age"] = 28
sample_dict["new_field"] = "새로운 정보를 추가 합니다"

# TypedDict
typed_dict["age"] = 28
type_dict["new_field"] = "새로운 정보를 추가 합니다"

NameError: name 'type_dict' is not defined

In [4]:
print(sample_dict)

{'name': '김영희', 'age': 28, 'job': 'ai engineer', 'new_field': '새로운 정보를 추가 합니다'}


In [5]:
print(typed_dict)

{'name': '김영희', 'age': 28, 'job': 'ai engineer'}


## Annotated

- 개요 
    - python의 `typing` 모듈에서 제공하는 기능 
    - 기본 타입 힌트에 메타데이터(설명, 조건 등)을 덧붙일 수 있게 해주는 도구 
    - 메타데이터는 pydantic, FastAPI 등 다양한 프레임워크에서 활용 (검증, 문서화 등)

- `Annotated`를 사용하는 주요 이유 

1. 추가 정보 제공(타입 힌트)
    - 타입 힌트에 추가적인 정보를 포함. 이는 코드를 읽는 사람이나 도구에 더 많은 Context 제공
    - 코드에 대한 추가 설명을 타입 힌트에 직접 포함시킬 수 있음
        - `name: Annotated[str, "이름"]`
        - `age: Annotated[int, "나이"]`

2. 문서화 
    - 코드 자체에 추가 설명을 포함시켜 문서화 형태로 관리 가능 

3. 유효성 검사 
    - 특정 라이브러리(`pydantic`)와 함께 사용하여 유효성 검사 수행 
        - `pydantic` : 데이터 유형성 검증, 데이터 파싱을 쉽게 처리할 수 있도록 도와주는 라이브러리 
        - 타입 힌트를 기반으로 데이터 검증 및 오류 방지 

In [ ]:
from typing import Annotated, List
from pydantic import Field, BaseModel, ValidationError

In [ ]:
class Student(BaseModel):
    id: Annotated[str, Field(..., description="학생ID")]
    name: Annotated[str, Field(..., min_length=3, max_length=50, description="이름")]
    age: Annotated[int, Field(gt=23, lt=31, description="나이(24~30세)")]
    skills: Annotated[
        List[str], Field(min_items=1, max_items=10, description="보유기술(1~10개)")
    ]

In [ ]:
try:
    valid_student = Student(
        id="skala-0001",
        name="김영희",
        age=26,
        skills=["Python", "Java"],
    )
    print("is_relevant: ", valid_student)

except ValidationError as e:
    print("not_relevant: ", e)

is_relevant:  id='skala-0001' name='김영희' age=26 skills=['Python', 'Java']


In [ ]:
# 유효하지 않은 케이스로 시도
try:
    valid_student = Student(
        id=1,
        name="영희",
        age=21,
        skills="Python",
    )
    print("is_relevant: ", valid_student)

except ValidationError as e:
    print("not_relevant: ")
    for error in e.errors():
        print(f"- {error['loc'][0]}: {error['msg']}")

not_relevant: 
- id: Input should be a valid string
- name: String should have at least 3 characters
- age: Input should be greater than 23
- skills: Input should be a valid list


## add_messages

1. 주요 기능
   - 두 개의 메시지 리스트를 병합
   - 기본적으로 "append-only" 상태 유지
   - 동일한 ID를 가진 메시지가 있을 경우, 새 메시지로 기존 메시지를 대체

2. 동작 방식
   - `right`의 메시지 중 `left`에 동일한 ID를 가진 메시지가 있으면, `right`의 메시지로 대체
   - 그 외의 경우 `right`의 메시지가 `left`에 추가

3. 매개변수
   - `left` (Messages): 기본 메시지 리스트
   - `right` (Messages): 병합할 메시지 리스트 또는 단일 메시지

4. 반환값
   - `Messages`: `right`의 메시지들이 `left`에 병합된 새로운 메시지 리스트

In [11]:
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.graph import add_messages

In [15]:
message1 = [HumanMessage(content="안녕하세요?", id="ID-001")]
message2 = [AIMessage(content="반갑습니다.", id="ID-002")]

result = add_messages(message1, message2)
print(result)

[HumanMessage(content='안녕하세요?', additional_kwargs={}, response_metadata={}, id='ID-001'), AIMessage(content='반갑습니다.', additional_kwargs={}, response_metadata={}, id='ID-002')]


In [16]:
message1 = [HumanMessage(content="안녕하세요?", id="ID-001")]
message2 = [AIMessage(content="반갑습니다.", id="ID-001")]

result = add_messages(message1, message2)
print(result)

[AIMessage(content='반갑습니다.', additional_kwargs={}, response_metadata={}, id='ID-001')]


-----
** End of Documents **